In [2]:
# Connect to Claude
import dspy
import openai

openai.api_key = "sk-foobar"
#CLAUDE_API_KEY = "sk-ant-foobar"

# `ollama run llama3.1`

gpt4 = dspy.OpenAI(model="gpt-4-1106-preview", max_tokens=4000, model_type="chat")


#claude = dspy.Claude(model="claude-3-opus-20240229", api_key=CLAUDE_API_KEY)

ollama_llama3 = dspy.OllamaLocal(model="llama3.1")

lms = [
    {"name": "GPT-4", "lm": gpt4},
    {"name": "Llama 3.1", "lm": ollama_llama3}
]

#lms.append({"name": "Claude Opus", "lm": claude})

dspy.settings.configure(lm=ollama_llama3)

connection_prompt = "Please say something interesting about Database Systems intended to impress me with your intelligence."

for lm_dict in lms:
    lm, name = lm_dict["lm"], lm_dict["name"]
    with dspy.context(lm=lm):
        print(f"\033[91mResult for {name}\n")
        print(f"\033[0m{lm(connection_prompt)[0]}\n")

Result for GPT-4

Database systems are the backbone of modern information management, enabling complex operations and analytics that drive decision-making across various industries. One particularly impressive aspect of database systems is their use of sophisticated algorithms for concurrency control, ensuring that multiple transactions can occur simultaneously without compromising data integrity.

For instance, the implementation of Multi-Version Concurrency Control (MVCC) in databases like PostgreSQL allows for each user connected to the database to see a snapshot of the database at a particular point in time. This means that readers do not block writers and writers do not block readers, providing highly concurrent access with minimal locking overhead. MVCC achieves this by keeping multiple versions of a data item, which elegantly solves the problem of maintaining consistency without sacrificing performance.

Furthermore, database systems often employ advanced indexing techniques, su

In [3]:
# DSPy Program

In [4]:
class GenerateResponse(dspy.Signature):
    """Follow the task_instructions (Input Field) and generate the response (Output Field) according to the output format given by response_format (Input Field). You will be given references from (Task-Specific Input Field).
VERY IMPORTANT!! ONLY OUTPUT JSON AND NOTHING ELSE!! THIS IS EXTEMELY IMPORTANT AND YOUR OUTPUT WILL BE USELESS OTHERWISE! DO NOT START YOUR OUTPUT WITH ```json OR END IT WITH ```! ONLY OUTPUT THE KEYS GIVEN BY THE response_format (Input Field)!!
    """

    task_instructions = dspy.InputField(desc="(Input Field)")
    response_format = dspy.InputField(desc="(Input Field)")
    references = dspy.InputField(desc="Task-Specific Input Field")
    response = dspy.OutputField(desc="(Output Field)")

In [5]:
# GenerateAnswersWithConfidence task

task_instructions = """
Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.
"""
response_format = "JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}"

In [6]:
generate_response = dspy.Predict(GenerateResponse)

example = generate_response(
    task_instructions=task_instructions,
    response_format=response_format,
    references="What is the atomic number of oxygen?"
).response

print(example)

{
  "answer": "8",
  "confidence": 5,
  {
    "answer": "The atomic number of oxygen is 8.",
    "confidence": 5
  }
}


In [7]:
# Load Data

In [8]:
import json

filename = "./wiki-answerable-questions.json"
with open(filename, 'r') as json_file:
    json_data = json.load(json_file)

# NEED TO PARSE THESE

data = []
for i in range(len(json_data)):
    # parse into `references` for the `inputField`
    context, question = json_data[i]["abstract"], json_data[i]["answerable_question"]
    references = f"{{'question': '{question}', 'context': '{context}'}}"
    data.append(dspy.Example(
        task_instructions=task_instructions,
        response_format=response_format,
        references=references).with_inputs("task_instructions", "response_format", "references"))

# 56 abstract-question pairs
trainset, devset, testset = data[:40], data[40:45], data[45:] # ~ 70% train | 10% dev | 20% test

print(len(data))

data[0]

56


Example({'task_instructions': '\nAssess the context and answer the question as well as a rating on how certain you are that your answer is correct.\n', 'response_format': 'JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}', 'references': "{'question': 'What was the purpose of Explorer 20?', 'context': 'Explorer 20, also known Ionosphere Explorer-A, IE-A, S-48, TOPSI and Topside Explorer, was a NASA satellite launched as part of Explorer program. Its purpose was two-fold: long-term investigation of the ionosphere from above, and in situ investigation of ion concentrations and temperatures.'}"}) (input_keys={'task_instructions', 'response_format', 'references'})

# Load JSON Metric

In [9]:
import json

def is_valid_json_output(output):
    try:
        parsed = json.loads(output)
        return isinstance(parsed.get("answer"), str) and isinstance(parsed.get("confidence"), int) and 0 <= parsed["confidence"] <= 5
    except json.JSONDecodeError:
        return False

# Could put types here -- would love to test one prompt for all types eventually
def JSON_Metric(gold, pred, trace=None):
    return is_valid_json_output(pred.response)

In [10]:
is_valid_json_output('{"answer": "Blue", "confidence": 3}')

True

In [11]:
is_valid_json_output("Sure I can help you with that!")

False

In [14]:
# Evaluate Uncompiled

In [15]:
from dspy.evaluate.evaluate import Evaluate

devset_evaluator = Evaluate(devset=devset, num_threads=1, display_progress=True)

print("Evaluating on devset...")
uncompiled_score = devset_evaluator(generate_response, metric=JSON_Metric)

print(uncompiled_score)

testset_evaluator = Evaluate(devset=testset, num_threads=1, display_progress=True)

print("Evaluating on testset...")
uncompiled_test_score = testset_evaluator(generate_response, metric=JSON_Metric)

print(uncompiled_test_score)

Evaluating on devset...
Average Metric: 0 / 5  (0.0): 100%|███████████████| 5/5 [00:11<00:00,  2.23s/it]
0.0
Evaluating on testset...
Average Metric: 2 / 11  (18.2): 100%|███████████| 11/11 [00:22<00:00,  2.06s/it]
18.18


# COPRO

In [16]:
from dspy.teleprompt import COPRO

COPRO_teleprompter = COPRO(prompt_model=gpt4,
                          metric=JSON_Metric,
                          breadth=10,
                          depth=2,
                          init_temperature=0.7,
                          verbose=True,
                          track_stats=True)
kwargs = dict(num_threads=1, display_progress=True, display_table=5)

COPRO_compiled_generate_response = COPRO_teleprompter.compile(generate_response, trainset=trainset, eval_kwargs=kwargs)




You are an instruction optimizer for large language models. I will give you a ``signature`` of fields (inputs and outputs) in English. Your task is to propose an instruction that will lead a good language model to perform the task well. Don't be afraid to be creative.

---

Follow the following format.

Basic Instruction: The initial instructions before optimization
Proposed Instruction: The improved instructions for the language model
Proposed Prefix For Output Field: The string at the end of the prompt, which will help the model start solving the task

---

Basic Instruction: Follow the task_instructions (Input Field) and generate the response (Output Field) according to the output format given by response_format (Input Field). You will be given references from (Task-Specific Input Field). VERY IMPORTANT!! ONLY OUTPUT JSON AND NOTHING ELSE!! THIS IS EXTEMELY IMPORTANT AND YOUR OUTPUT WILL BE USELESS OTHERWISE! DO NOT START YOUR OUTPUT WITH ```json OR END IT WITH ```! ONLY OUTPUT T

,task_instructions,response_format,references,response,JSON_Metric
0,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'What was the purpose of Explorer 20?', 'context': 'Explorer 20, also known Ionosphere Explorer-A, IE-A, S-48, TOPSI and Topside Explorer, was a NASA satellite...","{ ""answer"": ""The purpose of Explorer 20 was to investigate the ionosphere from above and conduct in-situ measurements of ion concentrations and temperatures."", ""confidence"": 5...",✔️ [True]
1,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'What does MAP4K3 encode in humans?', 'context': 'Mitogen-activated protein kinase kinase kinase kinase 3 is a protein that in humans is encoded by the...","{ ""answer"": ""a protein"", ""confidence"": 5, ""answer"": ""Mitogen-activated protein kinase kinase kinase kinase 3"", ""confidence"": 5 }",✔️ [True]
2,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'In which year was Heggholmen Lighthouse automated?', 'context': 'Heggholmen Lighthouse (Norwegian: Heggholmen fyr) is a coastal lighthouse located in the Oslofjord, in the municipality...","{ ""answer"": ""1972"", ""confidence"": 5, ""answer"": ""1827"", ""confidence"": 5 }",✔️ [True]
3,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'When did Antonio Nachura retire from his position as Associate Justice?', 'context': 'Antonio Eduardo Bermejo Nachura (June 13, 1941 – March 13, 2022) was...","{ ""answer"": ""June 13, 2011"", ""confidence"": 5, ""answer"": ""He took his oath of office as Associate Justice on February 7, 2007"", ""confidence"": 4 }",✔️ [True]
4,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'What position does Roman Gergel play in football?', 'context': 'Roman Gergel (born 22 February 1988) is a Slovak professional footballer who plays as a...","{ ""answer"": ""forward"", ""confidence"": 5, ""answer"": ""forward"", ""confidence"": 5 }",✔️ [True]





You are an instruction optimizer for large language models. I will give you a ``signature`` of fields (inputs and outputs) in English. Your task is to propose an instruction that will lead a good language model to perform the task well. Don't be afraid to be creative.

---

Follow the following format.

Basic Instruction: The initial instructions before optimization
Proposed Instruction: The improved instructions for the language model
Proposed Prefix For Output Field: The string at the end of the prompt, which will help the model start solving the task

---

Basic Instruction: Follow the task_instructions (Input Field) and generate the response (Output Field) according to the output format given by response_format (Input Field). You will be given references from (Task-Specific Input Field). VERY IMPORTANT!! ONLY OUTPUT JSON AND NOTHING ELSE!! THIS IS EXTEMELY IMPORTANT AND YOUR OUTPUT WILL BE USELESS OTHERWISE! DO NOT START YOUR OUTPUT WITH ```json OR END IT WITH ```! ONLY OUTPUT T

,task_instructions,response_format,references,response,JSON_Metric
0,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'What was the purpose of Explorer 20?', 'context': 'Explorer 20, also known Ionosphere Explorer-A, IE-A, S-48, TOPSI and Topside Explorer, was a NASA satellite...","{ ""answer"": ""The purpose of Explorer 20 was to investigate the ionosphere from above and conduct in-situ measurements of ion concentrations and temperatures."", ""confidence"": 5...",✔️ [True]
1,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'What does MAP4K3 encode in humans?', 'context': 'Mitogen-activated protein kinase kinase kinase kinase 3 is a protein that in humans is encoded by the...","{ ""answer"": ""A protein"", ""confidence"": 5, ""answer"": ""MAP4K3"", ""confidence"": 5 }",✔️ [True]
2,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'In which year was Heggholmen Lighthouse automated?', 'context': 'Heggholmen Lighthouse (Norwegian: Heggholmen fyr) is a coastal lighthouse located in the Oslofjord, in the municipality...","{ ""answer"": ""1972"", ""confidence"": 5, ""answer"": ""1827"", ""confidence"": 5 }",✔️ [True]
3,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'When did Antonio Nachura retire from his position as Associate Justice?', 'context': 'Antonio Eduardo Bermejo Nachura (June 13, 1941 – March 13, 2022) was...","{ ""answer"": ""June 13, 2011"", ""confidence"": 5, ""answer"": ""He took his oath of office as Associate Justice on February 7, 2007."", ""confidence"": 4 }",✔️ [True]
4,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'What position does Roman Gergel play in football?', 'context': 'Roman Gergel (born 22 February 1988) is a Slovak professional footballer who plays as a...","{ ""answer"": ""forward"", ""confidence"": 5, ""answer"": ""forward"", ""confidence"": 5 }",✔️ [True]





You are an instruction optimizer for large language models. I will give you a ``signature`` of fields (inputs and outputs) in English. Your task is to propose an instruction that will lead a good language model to perform the task well. Don't be afraid to be creative.

---

Follow the following format.

Basic Instruction: The initial instructions before optimization
Proposed Instruction: The improved instructions for the language model
Proposed Prefix For Output Field: The string at the end of the prompt, which will help the model start solving the task

---

Basic Instruction: Follow the task_instructions (Input Field) and generate the response (Output Field) according to the output format given by response_format (Input Field). You will be given references from (Task-Specific Input Field). VERY IMPORTANT!! ONLY OUTPUT JSON AND NOTHING ELSE!! THIS IS EXTEMELY IMPORTANT AND YOUR OUTPUT WILL BE USELESS OTHERWISE! DO NOT START YOUR OUTPUT WITH ```json OR END IT WITH ```! ONLY OUTPUT T

...




You are an instruction optimizer for large language models. I will give some task instructions I've tried, along with their corresponding validation scores. The instructions are arranged in increasing order based on their scores, where higher scores indicate better quality.

Your task is to propose a new instruction that will lead a good language model to perform the task even better. Don't be afraid to be creative.

---

Follow the following format.

Attempted Instructions: ${attempted_instructions}
Proposed Instruction: The improved instructions for the language model
Proposed Prefix For Output Field: The string at the end of the prompt, which will help the model start solving the task

---

Attempted Instructions:
[1] «Instruction #1: Proposed Instruction: Read the task_instructions (Input Field) carefully. Utilize the information provided in the Task-Specific Input Field to generate a precise response that adheres to the structure specified by the response_format (Input Field). 

,task_instructions,response_format,references,response,JSON_Metric
0,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'What was the purpose of Explorer 20?', 'context': 'Explorer 20, also known Ionosphere Explorer-A, IE-A, S-48, TOPSI and Topside Explorer, was a NASA satellite...","Here is the JSON output: ``` { ""answer"": ""long-term investigation of the ionosphere from above, and in situ investigation of ion concentrations and temperatures"", ""confidence"":...",False
1,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'What does MAP4K3 encode in humans?', 'context': 'Mitogen-activated protein kinase kinase kinase kinase 3 is a protein that in humans is encoded by the...","{ ""answer"": ""A protein that in humans is encoded by the MAP4K3 gene."", ""confidence"": 5, ""alternative_answer"": null }",✔️ [True]
2,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'In which year was Heggholmen Lighthouse automated?', 'context': 'Heggholmen Lighthouse (Norwegian: Heggholmen fyr) is a coastal lighthouse located in the Oslofjord, in the municipality...","Here is the JSON output: ``` { ""answer"": ""1972"", ""confidence"": 5, ""answer"": ""1827"", ""confidence"": 5 } ```",False
3,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'When did Antonio Nachura retire from his position as Associate Justice?', 'context': 'Antonio Eduardo Bermejo Nachura (June 13, 1941 – March 13, 2022) was...","{""answer"": ""June 13, 2011"", ""confidence"": 5}",✔️ [True]
4,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'What position does Roman Gergel play in football?', 'context': 'Roman Gergel (born 22 February 1988) is a Slovak professional footballer who plays as a...","{ ""answer"": ""forward"", ""confidence"": 5, { ""answer"": ""forward"", ""confidence"": 5 } }",False





You are an instruction optimizer for large language models. I will give some task instructions I've tried, along with their corresponding validation scores. The instructions are arranged in increasing order based on their scores, where higher scores indicate better quality.

Your task is to propose a new instruction that will lead a good language model to perform the task even better. Don't be afraid to be creative.

---

Follow the following format.

Attempted Instructions: ${attempted_instructions}
Proposed Instruction: The improved instructions for the language model
Proposed Prefix For Output Field: The string at the end of the prompt, which will help the model start solving the task

---

Attempted Instructions:
[1] «Instruction #1: Proposed Instruction: Read the task_instructions (Input Field) carefully. Utilize the information provided in the Task-Specific Input Field to generate a precise response that adheres to the structure specified by the response_format (Input Field). 

,task_instructions,response_format,references,response,JSON_Metric
0,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'What was the purpose of Explorer 20?', 'context': 'Explorer 20, also known Ionosphere Explorer-A, IE-A, S-48, TOPSI and Topside Explorer, was a NASA satellite...","{ ""answer"": ""The purpose of Explorer 20 was to investigate the ionosphere from above and conduct in-situ measurements of ion concentrations and temperatures."", ""confidence"": 5...",✔️ [True]
1,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'What does MAP4K3 encode in humans?', 'context': 'Mitogen-activated protein kinase kinase kinase kinase 3 is a protein that in humans is encoded by the...","{""answer"": ""a protein"", ""confidence"": 5, {""answer"": ""Mitogen-activated protein kinase kinase kinase kinase 3"", ""confidence"": 5}}",False
2,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'In which year was Heggholmen Lighthouse automated?', 'context': 'Heggholmen Lighthouse (Norwegian: Heggholmen fyr) is a coastal lighthouse located in the Oslofjord, in the municipality...","Here is the output in the specified format: { ""answer"": ""1972"", ""confidence"": 5, ""answer"": ""1827"", ""confidence"": 5 }",False
3,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'When did Antonio Nachura retire from his position as Associate Justice?', 'context': 'Antonio Eduardo Bermejo Nachura (June 13, 1941 – March 13, 2022) was...","Here is the output in the specified format: ```json { ""answer"": ""June 13, 2011"", ""confidence"": 5, { ""answer"": ""June 13, 2011"", ""confidence"": 5 } }...",False
4,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'What position does Roman Gergel play in football?', 'context': 'Roman Gergel (born 22 February 1988) is a Slovak professional footballer who plays as a...","{ ""answer"": ""forward"", ""confidence"": 5, ""answer"": ""forward"", ""confidence"": 5 }",✔️ [True]





You are an instruction optimizer for large language models. I will give some task instructions I've tried, along with their corresponding validation scores. The instructions are arranged in increasing order based on their scores, where higher scores indicate better quality.

Your task is to propose a new instruction that will lead a good language model to perform the task even better. Don't be afraid to be creative.

---

Follow the following format.

Attempted Instructions: ${attempted_instructions}
Proposed Instruction: The improved instructions for the language model
Proposed Prefix For Output Field: The string at the end of the prompt, which will help the model start solving the task

---

Attempted Instructions:
[1] «Instruction #1: Proposed Instruction: Read the task_instructions (Input Field) carefully. Utilize the information provided in the Task-Specific Input Field to generate a precise response that adheres to the structure specified by the response_format (Input Field). 

In [19]:
from dspy.evaluate.evaluate import Evaluate

devset_evaluator = Evaluate(devset=devset, num_threads=1, display_progress=True)

print("Evaluating on devset...")
compiled_score = devset_evaluator(COPRO_compiled_generate_response, metric=JSON_Metric)

print(uncompiled_score)

testset_evaluator = Evaluate(devset=testset, num_threads=1, display_progress=True)

print("Evaluating on testset...")
compiled_test_score = testset_evaluator(COPRO_compiled_generate_response, metric=JSON_Metric)

print(uncompiled_test_score)

Evaluating on devset...
Average Metric: 4 / 5  (80.0): 100%|██████████████| 5/5 [00:13<00:00,  2.75s/it]
80.0
Evaluating on testset...
Average Metric: 11 / 11  (100.0): 100%|█████████| 11/11 [00:13<00:00,  1.21s/it]
100.0


In [21]:
COPRO_compiled_generate_response.save("COPRO-Compiled-JSON-Mode.json") 

In [22]:
import json

with open("COPRO-Compiled-JSON-Mode.json", "r") as file:
    data = json.load(file)
    
print(json.dumps(data, indent=2))

{
  "lm": null,
  "traces": [],
  "train": [],
  "demos": [],
  "signature_instructions": "Proposed Instruction: Carefully read the task_instructions provided in the Input Field and create a response that adheres strictly to the structure specified by the response_format Input Field. Refer to the information from the Task-Specific Input Field as needed to accurately complete the task. Remember, the output must be in JSON format, containing only the keys indicated by the response_format. Do not include any additional characters or markdown formatting such as ``` before or after the JSON object.",
  "signature_prefix": "Output JSON:"
}
